# Procesamiento del Lenguaje Natural - Trabajo Práctico N°1 - 2024

- Asad, Gonzalo (A-4595/1)
- Castells, Sergio (C-7334/2)

---

## Preparación del Entorno

Instalación de librerías.

In [ ]:
!pip install transformers sentence_transformers
!pip install gdown
!pip install beautifulsoup4
!pip install deep_translator
!pip install gliner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 4.6 MB/s eta 0:00:00


Carga de librerías.

In [ ]:
import gdown
import pandas as pd
import numpy as np
from typing import Any

import requests
from bs4 import BeautifulSoup
import time
import re
import warnings
warnings.filterwarnings('ignore')

import torch
from sentence_transformers import SentenceTransformer, util
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, jaccard_score
from transformers import BertTokenizer, BertModel
from deep_translator import GoogleTranslator, DeeplTranslator
from gliner import GLiNER

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


## Web-Scrapping

*No ejecutar esta sección. Es usada únicamente para generar la base de datos de
libros basado en el Proyecto Gutenberg. Se deja aquí únicamente para documentar
el proceso de web-scrapping para conseguirla, la base de datos será descargada
automáticamente desde la nube junto con el resto desde el programa principal.*

In [ ]:
class RequestError(Exception):
    '''Excepción personalizada para errores en las solicitudes.'''
    pass

def getValidation(url: str, retries: int = 3, backoff_factor: float = 0.5) -> str:
  '''Valida la petición GET a la URL y en caso exitoso retorna la respuesta.
     Implementa reintentos con backoff exponencial para manejar errores de timeout.'''
  for i in range(retries):
      try:
          # Generación de la solicitud GET a la URL
          response = requests.get(url)
          # Verificación de si la respuesta tiene un código de error
          response.raise_for_status()
      except requests.exceptions.HTTPError as http_err:
          if response.status_code == 504 and i < retries - 1:
              time.sleep(backoff_factor * (2 ** i))  # Se espera antes de reintentar
              print(f"Reintentando {url} (intento {i + 1}/{retries})")
              continue # Se reintenta la solicitud
          raise RequestError(f"Error HTTP: {http_err}")
      except requests.exceptions.RequestException as err:
          raise RequestError(f"Error en la solicitud: {err}")
      else:
          # Si no hay error se retorna la respuesta
          return response

In [ ]:
# URL de la página de donde quiero extraer la info
url = "https://www.gutenberg.org/browse/scores/top1000.php#books-last1"

# Petición GET a la URL.
response = getValidation(url)

# Parseo del contenido HTML de la página utilizando BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# La estructura HTML en cuestión contiene varias listas ordenadas <ol>, cada una de las cuales agrupa 1000 items <li>
# Se selecciona la primera <ol> en el documento
first_ol = soup.find("ol")

# Se seleccionan todos los <li> dentro de ese primer <ol>
books = first_ol.find_all("li") if first_ol else []

# Si se quisiera encontrar todos los <ol> y <li>, se podría usar lo siguiente
# books = soup.select("ol li")

# Definición de lista para almacenar los datos de cada libro
books_data = []

# Extracción de información de cada libro. Se pretende extraer Título, Autor, Género, Idioma y Resumen
for book in books:
    # Inspeccionando el documento HTML se puede ver que cada ítem de la lista ordenada tiene el siguiente formato:
    # <li><a href="/ebooks/84">Frankenstein; Or, The Modern Prometheus by Mary Wollstonecraft Shelley (8508)</a></li>
    # por ende, de ahí se pueden obtener el nombre del libro y el autor (cadena de texto con separador "by"), además del link
    # a la página web donde hay más info del mismo

    title_author = book.get_text()
    link = book.find("a")["href"] if book.find("a") else None

    # Separación de Título y Autor
    if " by " in title_author:
        title, author = title_author.split(" by ", 1)
    else:
        # En caso de que no exista " by " en la cadena de texto, se considera que el autor es desconocido
        title, author = title_author, "Unknown"

    # El nombre del autor viene acompañado de un número entre paréntesis. Se elimina usando una RegEx.
    author = re.sub(r"\s*\(\d+\)$", "", author)

    # Limpieza de los espacios en blanco al principio y al final del texto.
    title = title.strip()
    author = author.strip()

    # Búsqueda de info en las páginas exclusivas de cada libro
    book_url = f"https://www.gutenberg.org{link}"

    # Petición GET a la URL.
    response = getValidation(book_url)

    # Parseo del contenido HTML de la página utilizando BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')

    # Inspeccionando el documento HTML se observa que la información acerca del libro se
    # encuentra en una estructura de tipo tabla donde por cada fila <tr> se tiene una columna a modo de
    # encabezado <th> y otra columna donde se almacenan los datos <td>

    # Extracción del Género
    # La información de género literiario se encuentra en una estructura como la siguiente:
    #       <tr>
    #         <th>Subject</th>
    #         <td property="dcterms:subject" datatype="dcterms:LCSH">
    #           <a class="block" href="/ebooks/subject/36"> Science fiction </a>
    #         </td>
    #       </tr>
    # Sin embargo hay libros que tienen más de un género y libros que no tienen ninguno

    # Extracción del texto de la etiqueta <a> correspondiente a la primer fila que contenga el encabezado "Subject" y que en el campo
    # de datos posea al atributo property con el valor 'dcterms:subject'.
    # En caso de no haberse detectado ninguno completo con None. Agrego estas validacion porque sino se genera
    # un error al no encontrar elementos.
    subjects = soup.select("tr:has(th:contains('Subject')) td[property='dcterms:subject'] a")[0].get_text(strip=True) if soup.select("tr th:contains('Subject') + td") else None

    # Extracción el Idioma
    # La información de idioma se encuentra en una estructura como la siguiente:
    #      <tr property="dcterms:language" datatype="dcterms:RFC4646" itemprop="inLanguage" content="en">
    #      <th>Language</th>
    #      <td>Inglés</td>
    #      </tr>

    # Se selecciona la primera celda de datos <td> correspondiente al encabezado "Language"
    # [soup.select("tr th:contains('Language') + td") devuelve
    # una lista con todos los elementos <td> encontrados que contienen el idioma (aunque debería ser solo uno)]
    # Agrego la validación (if...) porque sino se genera un error al no encontrar elementos.
    language = soup.select("tr th:contains('Language') + td")[0].get_text(strip=True) if soup.select("tr th:contains('Language') + td") else None

    # Extracción del Resumen
    # La información del resumen se encuentra en una estructura como la siguiente:
    #      <tr>
    #      <th>Summary</th>
    #      <td>
    #      "Frankenstein; Or, The Modern Prometheus" by Mary Wollstonecraft Shelley is a novel written in the early 19th century.
    #       The story explores themes of ambition, the quest for knowledge, and the consequences of man's hubris through the experiences
    #        of Victor Frankenstein and the monstrous creation of his own making...
    #      </td>
    #      </tr>

    # Se selecciona la primera celda de datos <td> correspondiente al encabezado "Summary"
    # Se agrega la validación (if...) porque sino se genera un error al no encontrar elementos.
    summary = soup.select("tr th:contains('Summary') + td")[0].get_text(strip=True) if soup.select("tr th:contains('Summary') + td") else None

    # Al final de cada resumen se encuentra el texto "(This is an automatically generated summary.)". Se elimina.
    summary = summary.replace("(This is an automatically generated summary.)", "").strip() if summary else None

    # Creación de diccionario con la información del libro y lo agrego a la lista.
    book_info = {
        "title": title,
        "author": author,
        "subjects": subjects,
        "language": language,
        "summary": summary
    }

    books_data.append(book_info)


# Creación de DataFrame con la información de los libros
books_df = pd.DataFrame(books_data)

# Guardado el DataFrame en un archivo CSV.
books_df.to_csv("books_database.csv", index=False, encoding="utf-8")

## Clasificación del Estado de Ánimo

Función de clasificación de estado de ánimo usando un modelo kNN de clasificación (EN DESUSO).

In [ ]:
def emotionClassifierModelkNN(df: pd.DataFrame, production: bool = False) -> KNeighborsClassifier:
  '''
  Esta función genera un modelo de kNN de clasificación
  de estados de ánimo.
  La función recibe como argumentos:
    - df: base de datos de estados de ánimo clasificados
    - production: si es False, se calculan y muestras las métricas de prueba
  La función devuelve un modelo de kNN de clasificación.
  '''
  # Carga del modelo desde HuggingFace https://huggingface.co/sentence-transformers/all-mpnet-base-v2
  # model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
  model = SentenceTransformer('distiluse-base-multilingual-cased-v1')
  # model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')
  # model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')

  # Generación de estructura con el dataset de estados de ánimo
  dataset_est_animo = []
  for row in df.itertuples():
      dataset_est_animo.append((row.label, row.estado_animo))

  # Preparación de X e y
  X = [text.lower() for label, text in dataset_est_animo]
  y = [label for label, text in dataset_est_animo]

  # División del dataset
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

  # Obtención los embeddings de BERT para los conjuntos de entrenamiento
  X_train_vectorized = model.encode(X_train)

  # Creación y entrenamiento del modelo de kNN

  # Definición los parámetros
  param_grid = {'n_neighbors': np.arange(1, 10),
                'weights': ['uniform', 'distance'],
                'p': [1, 2],
                'metric': ['minkowski'],#, 'manhattan'],
                'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
                'leaf_size': [10, 20, 30, 40]
                }

  # Creación el objeto GridSearchCV
  grid_search_cv = GridSearchCV(KNeighborsClassifier(), param_grid, cv=5)

  # Ajuste del modelo con los datos de entrenamiento
  grid_search_cv.fit(X_train_vectorized, y_train)

  # Obtención del mejor estimador
  modelo_kNN = grid_search_cv.best_estimator_
  best_params = grid_search_cv.best_params_

  # Entrenamiento del modelo kNN
  modelo_kNN.fit(X_train_vectorized, y_train)

  if not production:
    # Obtención de los embeddings de BERT para los conjuntos de prueba
    X_test_vectorized = model.encode(X_test)

    # Evaluación del modelo de Regresión Logística
    y_pred_kNN = modelo_kNN.predict(X_test_vectorized)
    acc_kNN = accuracy_score(y_test, y_pred_kNN)
    report_kNN = classification_report(y_test, y_pred_kNN, zero_division=1)

    print("\Exactitud kNN:", acc_kNN)
    print("Mejores parámetros:", best_params)
    print("Reporte de clasificación kNN:\n", report_kNN)

  return modelo_kNN

Función de clasificación de estado de ánimo usando un modelo de Regresión Logística.

In [ ]:
def emotionClassifierModel(df: pd.DataFrame, production: bool = False) -> LogisticRegression:
  '''
  Esta función genera un modelo de regresión logística de clasificación
  de estados de ánimo.
  La función recibe como argumentos:
    - df: base de datos de estados de ánimo clasificados
    - production: si es False, se calculan y muestras las métricas de prueba
  La función devuelve un modelo de regresión logística.
  '''
  # Carga del modelo desde HuggingFace https://huggingface.co/sentence-transformers/all-mpnet-base-v2
  # model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
  model = SentenceTransformer('distiluse-base-multilingual-cased-v1')
  # model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')
  # model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')

  # Generación de estructura con el dataset de estados de ánimo
  dataset_est_animo = []
  for row in df.itertuples():
      dataset_est_animo.append((row.label, row.estado_animo))

  # Preparación de X e y
  X = [text.lower() for label, text in dataset_est_animo]
  y = [label for label, text in dataset_est_animo]

  # División del dataset
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

  # Obtención de los embeddings de BERT para los conjuntos de entrenamiento
  X_train_vectorized = model.encode(X_train)

  # Creación y entrenamiento del modelo de Regresión Logística Multinomial
  modelo_LR = LogisticRegression(max_iter=1000, multi_class='multinomial', solver='lbfgs')
  modelo_LR.fit(X_train_vectorized, y_train)

  if not production:
    # Obtención de los embeddings de BERT para los conjuntos de prueba
    X_test_vectorized = model.encode(X_test)

    # Evaluación del modelo de Regresión Logística
    y_pred_LR = modelo_LR.predict(X_test_vectorized)
    acc_LR = accuracy_score(y_test, y_pred_LR)
    report_LR = classification_report(y_test, y_pred_LR, zero_division=1)

    print("\Exactitud Regresión Logística:", acc_LR)
    print("Reporte de clasificación Regresión Logística:\n", report_LR)

  return modelo_LR

Función de clasificación de estados de ánimo.

In [ ]:
def emotionClassifier(modelo_LR: LogisticRegression, estado_animo: str) -> str:
  '''
  Función que clasifica un estado de ánimo que recibe como argumento, retornando
  la clasificación a su salida.
  Recibe como argumentos:
   - modelo_LR: el modelo de clasificación
   - estado_animo: el estado de ánimo a clasificar
  '''
  # Carga del modelo desde HuggingFace https://huggingface.co/sentence-transformers/all-mpnet-base-v2
  # model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
  model = SentenceTransformer('distiluse-base-multilingual-cased-v1')
  # model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')
  # model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')

  # Adaptación del estado de ánimo
  estado_animo = [estado_animo.lower()]

  # Preprocesamiento y vectorización de las nuevas frases
  estado_animo_vectorized = model.encode(estado_animo)

  # Predicción con el modelo entrenado
  label = modelo_LR.predict(estado_animo_vectorized)

  # Impresión del estado de ánimo y su etiquetado
  # print(f"\nEstado de ánimo: '{estado_animo}'")
  # print(f"Clasificación predicha: {label}\n")

  return label

## Sistema de Recomendaciones

Función generadora de incrustaciones (embeddings).

In [ ]:
def embeddingsGenerator(df_imdb: pd.DataFrame, df_bgg: pd.DataFrame, df_gutenberg: pd.DataFrame) -> "Any":
  '''
  Esta función genera los embeddings de las bases de datos.
  '''
  # Carga del modelo preentrenado multilingüe
  modelo = SentenceTransformer('distiluse-base-multilingual-cased-v1')
  # modelo = SentenceTransformer('msmarco-MiniLM-L-6-v3')

  # Codificación de las bases de datos
  embeddings_imdb = modelo.encode(df_imdb.Director + ' ' + df_imdb.Actors + ' ' + df_imdb.Genre + ' ' + df_imdb.Description, convert_to_tensor=True)
  embeddings_bgg = modelo.encode(df_bgg.description, convert_to_tensor=True)
  embeddings_gutenberg = modelo.encode(df_gutenberg.summary, convert_to_tensor=True)

  return embeddings_imdb, embeddings_bgg, embeddings_gutenberg

Función generadora de recomendaciones.

In [ ]:
def recommendations(user_prompt: str, embeddings: Any, top: int, category: str, df: pd.DataFrame) -> None:
  '''
  Función que hace recomendaciones en base al prompt de un usuario.
  Recibe como argumentos:
   - user_prompt: la solicitud del usuario
   - embeddings: los embeddings de una base de datos
   - top: el número de recomendaciones que se quieren obtener
   - category: la categoría de la base de datos
   - df: el dataframe de la base de datos
  '''
  # Carga del modelo preentrenado multilingüe
  modelo = SentenceTransformer('distiluse-base-multilingual-cased-v1')

  # Traducción del prompt al inglés
  user_prompt_eng = GoogleTranslator(source='spanish', target='english').translate(user_prompt)
  #print(user_prompt_eng)
  user_prompt_embedded = modelo.encode(user_prompt_eng, convert_to_tensor=True)

  # Cálculo de puntuaciones de similitud
  puntuaciones_coseno = util.cos_sim(embeddings, user_prompt_embedded)
  # puntuaciones_coseno = jaccard_score(embeddings, user_prompt_embedded, average='micro')

  # Búsqueda de las puntuaciones de similitud más altas
  pares = []
  for i in range(len(puntuaciones_coseno)):
      pares.append({'index': i, 'score': puntuaciones_coseno[i][0]})

  # Ordenamiento de las puntuaciones en orden decreciente
  pares = sorted(pares, key=lambda x: x['score'], reverse=True)

  # El Código que sigue a continuación es para darle un look estético amigable a la salida

  # Creación de lista para DataFrame de recomendaciones
  recomendaciones = []
  for puesto, par in enumerate(pares[:top], start=1):
    i = par['index']
    if category == 'películas':
      recomendaciones.append([puesto, df.loc[i].Title, f"{par['score']:.4f}"])
    elif category == 'juegos de mesa':
      recomendaciones.append([puesto, df.loc[i].game_name, f"{par['score']:.4f}"])
    elif category == 'libros':
      recomendaciones.append([puesto, df.loc[i].title, f"{par['score']:.4f}"])

  # Conversión a DataFrame para usar estilo similar a pandas
  df_recomendaciones = pd.DataFrame(recomendaciones, columns=["Puesto", f"Sugerencias de {category.capitalize()}", "Puntuación de Similitud"])

  # Aplicación de estilo con fondo oscuro y texto blanco
  styled_table = df_recomendaciones.style.set_properties(
    **{'background-color': '#343a40', 'color': '#ffffff', 'border': '1px solid #dee2e6'}
  ).set_table_styles([
    {'selector': 'thead th', 'props': [('background-color', '#f8f9fa'), ('color', '#343a40'), ('font-weight', 'bold')]},
    {'selector': 'tr:hover', 'props': [('background-color', '#495057')]}  # Efecto hover más oscuro
  ])

  # Impresión de la tabla con estilo de pandas, sin índice
  display(HTML(f"<h4 style='color: #f8f9fa;'>Recomendaciones de {category.capitalize()}</h4>"))
  display(styled_table)

Función para reconocimiento de entidades nombradas.

In [ ]:
def labelsDetector(user_prompt: str) -> dict:
  '''
  Detecta determinadas etiquetas contenidas en un prompt de usuario.
  Parámetros:
   - user_prompt: la solicitud del usuario

  Retorno:
   - un diccionario donde cada clave es una etiqueta y su valor es
     una lista de todas las entidades detectadas para dicha etiqueta.
  '''
  # Carga del modelo preentrenado 'gliner_multi-v2.1' desde Hugging Face
  model = GLiNER.from_pretrained("urchade/gliner_multi-v2.1")

  # Se cambia el modelo a modo de evaluación, esto es útil para desactivar características específicas como dropout durante la inferencia
  model.eval()

  # Lista de etiquetas que el modelo intentará encontrar en el texto
  labels = ["actor", "director", "genre", "author"]

  # Predicción de entidades en el texto dado, utilizando las etiquetas especificadas y un umbral de 0.4
  entities = model.predict_entities(user_prompt, labels, threshold=0.4)

  # Inicialización de un diccionario para almacenar las etiquetas detectadas como claves y las entidades
  # correspondientes como valores en forma de lista (para manejar múltiples instancias de la misma etiqueta).
  labels_detected = {}

  # Iteración a través de las entidades reconocidas en el texto.
  for entity in entities:
    label = entity["label"]
    text = entity["text"]
    # Verificación de si la etiqueta ya existe en el diccionario
    if label in labels_detected:
        # Si existe, se añade el texto a la lista
        labels_detected[label].append(text)
    else:
        # Si no existe, se crea una nueva lista con el primer texto
        labels_detected[label] = [text]

  # Devolución del diccionario 'labels_detected', donde cada clave es una etiqueta y su valor es
  # una lista de todas las entidades detectadas para dicha etiqueta.
  return labels_detected

Función clasificadora de recomendaciones.

In [ ]:
def recommender(moodState: str, moodModel: Any, topicPhrase: str, embeddings: Any) -> None:
  '''
  Genera recomenciones de películas, libros y/o juegos de mesa en función
  del estado de ánimo y una frase temática suministrada por el usuario.

  Parámetros:
    - moodState: Frase o palábra asociada al estádo de ánimo.
    - moodModel: Modelo Clasificador de estados de ánimo.
    - topicPhrase: Frase o palábra asociada a la temática.
    - embeddings: Embeddings de las bases de datos.
  '''

  # Detección de Estado de Ánimo
  estado_animo = emotionClassifier(moodModel, moodState)

  # Definición de los conjuntos Película y Libro
  movie = {"actor", "director", "genre"}
  book = {"author"}

  # Embeddings
  embeddings_imdb = embeddings[0]
  embeddings_bgg = embeddings[1]
  embeddings_gutenberg = embeddings[2]

  # Etiquetas
  labels = labelsDetector(topicPhrase).keys()
  # print(f"Etiquetas: {labels}")

  # Recomendación de Películas
  if movie.intersection(labels) and not book.intersection(labels):
    match estado_animo:
      case "positivo":
        top_movies = 3
      case "neutro":
        top_movies = 4
      case "negativo":
        top_movies = 5

    recommendations(topicPhrase, embeddings_imdb, top_movies, 'películas', df_imdb)

  # Recomendación de Películas y Libros
  elif movie.intersection(labels) and book.intersection(labels):
    match estado_animo:
      case "positivo":
        top_movies = 3
        top_books = 3
      case "neutro":
        top_movies = 4
        top_books = 4
      case "negativo":
        top_movies = 5
        top_books = 5

    recommendations(topicPhrase, embeddings_imdb, top_movies, 'películas', df_imdb)
    recommendations(topicPhrase, embeddings_gutenberg, top_books, 'libros', df_gutenberg)

  #Recomendación de Libros
  elif book.intersection(labels) and not movie.intersection(labels):
    match estado_animo:
      case "positivo":
        top_books = 3
      case "neutro":
        top_books = 4
      case "negativo":
        top_books = 5

    recommendations(topicPhrase, embeddings_gutenberg, top_books, 'libros', df_gutenberg)

  # Recomendación de Películas, Juegos de Mesa y Libros
  else:
    match estado_animo:
      case "positivo":
        top_movies = 2
        top_books = 1
        top_games = 3
      case "neutro":
        top_movies = 1
        top_books = 3
        top_games = 2
      case "negativo":
        top_movies = 3
        top_books = 2
        top_games = 1

    recommendations(topicPhrase, embeddings_imdb, top_movies, 'películas', df_imdb)
    recommendations(topicPhrase, embeddings_gutenberg, top_books, 'libros', df_gutenberg)
    recommendations(topicPhrase, embeddings_bgg, top_games, 'juegos de mesa', df_bgg)

## Programa Principal

### Funciones adicionales

Función para carga de datasets.

In [ ]:
def getDataFrame(file_id: str, encoding: str ='utf-8', delimiter: str = ',', show_head: bool = False) -> pd.DataFrame:
  '''
  Crea un DataFrame a partir de un archivo CSV alojado en Google Drive.

  Parámetros:
    - file_id: ID del archivo (codificado según Google Drive).
    - encoding: Codificación de caracteres que se utilizará para leer el archivo.
    - delimiter: Especifica el carácter que separa los valores en el archivo.
    - show_head: Si es True, muestra las primeras filas del DataFrame.

  Retorno:
    - DataFrame: El DataFrame creado a partir del archivo.
  '''

  # Creación la URL de descarga
  download_url = f'https://drive.google.com/uc?id={file_id}'

  # Descarga del archivo
  output = 'file'
  gdown.download(download_url, output, quiet=True)

  df= pd.read_csv('file', encoding='utf-8', delimiter=',')

  if show_head:
    df.head()

  return df

Función para ejecutar el algoritmo de recomendación.

In [ ]:
def onRecommendButtonClicked(moodInput: str, topicInput: str, moodModel: Any, embeddings: Any, outputMessage: widgets.Output) -> None:
  '''
  Determina si el estado de ánimo y la frase descriptiva de un usuario
  es válida. Caso que alguno no sea válido se le solicita al usuario que ingrese
  lo que corresponde nuevamente. Caso contrario, se genera la recomendación.

  Parámetros:
  - moodInput: Widget de entrada de texto para el estado de ánimo.
  - topicInput: Widget de entrada de texto para la frase descriptiva.
  - moodModel: Modelo de clasificación de estados de ánimo.
  - embeddings: Embeddings de las bases de datos.
  - outputMessage: Widget de salida (Output) que muestra mensajes al usuario.
  '''
  clear_output(wait=True)
  mood_state = moodInput.value.strip()
  topic_phrase = topicInput.value.strip()

  with outputMessage:
    clear_output()
    if len(mood_state) < 3:
      print("❌ Error: Por favor, ingresa un estado de ánimo más descriptivo.")
    elif len(topic_phrase) < 10:
      print("❌ Error: La temática debe tener al menos 15 caracteres.")
    else:
      print("Generando recomendaciones...")
      # print(mood_state)
      # print(topic_phrase)
      recommender(mood_state, moodModel, topic_phrase, embeddings)

### Main

In [ ]:
# Main

# Estilo y títulos
title = widgets.HTML("<h2 style='color:darkblue;'>Clasificador de Recomendaciones Recreativas</h2>")
subtitle = widgets.HTML("<p>Este sistema te recomendará opciones de entretenimiento según tu estado de ánimo y preferencias temáticas.</p>")

# Preparación del Entorno

# Verificación de Existencia de DataFrames
df_status_label = widgets.HTML("<b style='color:darkgreen;'>Verificación de DataFrames</b>")
df_status_output = widgets.Output()

with df_status_output:
  clear_output()
  print("Verificando existencia de DataFrames necesarios...")
  try:
    df_est_animo
    print("✔ DataFrame de Estados de Ánimo cargado.")
  except NameError:
    print("⚠ Cargando DataFrame de Estados de Ánimo.")
    df_est_animo = getDataFrame('12p-Jc6huPeWWrLXSXpgGE-v4jbhNapxd')

  try:
    df_imdb
    print("✔ DataFrame de Películas cargado.")
  except NameError:
    print("⚠ Cargando DataFrame de Películas.")
    df_imdb = getDataFrame('1DGyMR5FV-zpRaNKsi2Pszuf46kxW9zso')
    df_imdb[['Description', 'Genre', 'Actors', 'Director']] = df_imdb[['Description', 'Genre', 'Actors', 'Director']].fillna('')
    df_imdb[['Description', 'Genre', 'Actors', 'Director']] = df_imdb[['Description', 'Genre', 'Actors', 'Director']].astype("string")

  try:
    df_bgg
    print("✔ DataFrame de Juegos de Mesa cargado.")
  except NameError:
    print("⚠ Cargando DataFrame de Juegos de Mesa.")
    df_bgg = getDataFrame('1eyoCL5m_snQaJNt3ZkYxeSwv_Jfq5PKQ')

  try:
    df_gutenberg
    print("✔ DataFrame de Libros cargado.")
  except NameError:
    print("⚠ Cargando DataFrame de Libros cargado.")
    df_gutenberg = getDataFrame('1Xr5jQS5ZIbclvADROQ5l5FTruC5q0V2s')
    df_gutenberg[['summary', 'subjects']] = df_gutenberg[['summary', 'subjects']].fillna('')
    df_gutenberg[['summary', 'subjects']] = df_gutenberg[['summary', 'subjects']].astype("string")


# Verificación de Existencia de Modelos y Embeddings
model_status_label = widgets.HTML("<b style='color:darkgreen;'>Verificación de Modelos y Embeddings</b>")
model_status_output = widgets.Output()

with model_status_output:
  clear_output()
  print("Verificando existencia de Modelos y Embeddings necesarios...")
  # Modelo Clasificador de Emociones
  try:
    mood_model
    print("✔ Modelo Clasificador de Emociones cargado.")
  except NameError:
    print("⚠ Cargando Modelo Clasificador de Emociones.")
    mood_model = emotionClassifierModel(df_est_animo, True)

  # Embeddings
  try:
    embeddings
    print("✔ Embeddings cargadas.")
  except NameError:
    print("⚠ Cargando Embeddings.")
    embeddings = embeddingsGenerator(df_imdb, df_bgg, df_gutenberg)


# Interacción con el usuario
mood_input = widgets.Text(placeholder="¿Cómo estas hoy?", layout=widgets.Layout(width='40%'))
topic_input = widgets.Text(placeholder="¿Qué temática te gustaría explorar?", layout=widgets.Layout(width='40%'))
input_section = widgets.VBox([
    widgets.Label("Estado de Ánimo:"), mood_input,
    widgets.Label("Temática de Interés:"), topic_input
])


# Mensaje de salida
output_message = widgets.Output()

# Botón de recomendación
recommend_button = widgets.Button(description="Obtener Recomendación", button_style='primary', layout=widgets.Layout(width='40%'))
recommend_button.on_click(lambda b: onRecommendButtonClicked(mood_input, topic_input, mood_model, embeddings, output_message))

# Layout final de la interfaz
app_layout = widgets.VBox([
    title,
    subtitle,
    widgets.Accordion(children=[df_status_output], titles=('Verificación de Datos',), layout=widgets.Layout(width='40%')),
    widgets.Accordion(children=[model_status_output], titles=('Verificación de Modelos y Embeddings',), layout=widgets.Layout(width='40%')),
    input_section,
    widgets.HBox([recommend_button]),
    output_message
])

# Mostrar la interfaz
display(app_layout)